Code for sections: 
- "Walc and other grade attributes"
- "Exploratory Data Analysis"

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
from collections import OrderedDict
from matplotlib import pyplot
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
url = "https://raw.githubusercontent.com/sarahyurick/datasets/master/alc_merged.csv"

data = pd.read_csv(url)

In [7]:
data.corr()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
age,1.000000,-0.119031,-0.133645,0.014530,0.001454,0.280021,-0.008453,-0.002414,0.094744,0.130655,0.069320,-0.013347,0.168313,-0.113206,-0.130408,-0.127446
Medu,-0.119031,1.000000,0.635156,-0.239830,0.104501,-0.163166,0.010211,-0.011450,0.022866,-0.018013,-0.036381,-0.002183,0.095703,0.208172,0.202776,0.182611
Fedu,-0.133645,0.635156,1.000000,-0.189146,0.045236,-0.170862,0.016171,0.009450,0.053071,-0.008774,0.032370,0.038402,0.068048,0.185636,0.156541,0.137159
traveltime,0.014530,-0.239830,-0.189146,1.000000,-0.066172,0.078820,-0.010459,-0.007536,0.039607,0.084199,0.060548,-0.045230,-0.049896,-0.071572,-0.111347,-0.073488
studytime,0.001454,0.104501,0.045236,-0.066172,1.000000,-0.125250,0.003935,-0.077845,-0.078264,-0.139678,-0.220301,-0.054130,-0.039015,0.172011,0.133655,0.097863
failures,0.280021,-0.163166,-0.170862,0.078820,-0.125250,1.000000,-0.068134,0.114463,0.060217,0.088939,0.084924,0.035372,0.087584,-0.368938,-0.360881,-0.359377
famrel,-0.008453,0.010211,0.016171,-0.010459,0.003935,-0.068134,1.000000,0.140165,0.091317,-0.069881,-0.092647,0.109476,-0.067647,0.060893,0.052807,0.061296
freetime,-0.002414,-0.011450,0.009450,-0.007536,-0.077845,0.114463,0.140165,1.000000,0.332688,0.117690,0.127746,0.088609,-0.042332,-0.040185,-0.061199,-0.058860
goout,0.094744,0.022866,0.053071,0.039607,-0.078264,0.060217,0.091317,0.332688,1.000000,0.252217,0.404503,0.002163,0.025046,-0.093786,-0.091030,-0.081691
Dalc,0.130655,-0.018013,-0.008774,0.084199,-0.139678,0.088939,-0.069881,0.117690,0.252217,1.000000,0.618431,0.065323,0.125517,-0.112935,-0.084944,-0.075212


In [8]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
age,1.000000,-0.119031,-0.133645,0.014530,0.001454,0.280021,-0.008453,-0.002414,0.094744,0.130655,0.069320,-0.013347,0.168313,-0.113206,-0.130408,-0.127446
Medu,-0.119031,1.000000,0.635156,-0.239830,0.104501,-0.163166,0.010211,-0.011450,0.022866,-0.018013,-0.036381,-0.002183,0.095703,0.208172,0.202776,0.182611
Fedu,-0.133645,0.635156,1.000000,-0.189146,0.045236,-0.170862,0.016171,0.009450,0.053071,-0.008774,0.032370,0.038402,0.068048,0.185636,0.156541,0.137159
traveltime,0.014530,-0.239830,-0.189146,1.000000,-0.066172,0.078820,-0.010459,-0.007536,0.039607,0.084199,0.060548,-0.045230,-0.049896,-0.071572,-0.111347,-0.073488
studytime,0.001454,0.104501,0.045236,-0.066172,1.000000,-0.125250,0.003935,-0.077845,-0.078264,-0.139678,-0.220301,-0.054130,-0.039015,0.172011,0.133655,0.097863
failures,0.280021,-0.163166,-0.170862,0.078820,-0.125250,1.000000,-0.068134,0.114463,0.060217,0.088939,0.084924,0.035372,0.087584,-0.368938,-0.360881,-0.359377
famrel,-0.008453,0.010211,0.016171,-0.010459,0.003935,-0.068134,1.000000,0.140165,0.091317,-0.069881,-0.092647,0.109476,-0.067647,0.060893,0.052807,0.061296
freetime,-0.002414,-0.011450,0.009450,-0.007536,-0.077845,0.114463,0.140165,1.000000,0.332688,0.117690,0.127746,0.088609,-0.042332,-0.040185,-0.061199,-0.058860
goout,0.094744,0.022866,0.053071,0.039607,-0.078264,0.060217,0.091317,0.332688,1.000000,0.252217,0.404503,0.002163,0.025046,-0.093786,-0.091030,-0.081691
Dalc,0.130655,-0.018013,-0.008774,0.084199,-0.139678,0.088939,-0.069881,0.117690,0.252217,1.000000,0.618431,0.065323,0.125517,-0.112935,-0.084944,-0.075212


In [9]:
drop_col = ['school', 'age', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime',
            'nursery']
data = data.drop(columns=drop_col)

# handle non-numeric features
struct_data = data.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)
# print(non_numeric_columns)
le = LabelEncoder()
for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

data = struct_data
data.columns

Index(['sex', 'address', 'studytime', 'failures', 'schoolsup', 'famsup',
       'paid', 'activities', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3'],
      dtype='object')

In [10]:
print(data['Dalc'].unique())
print(data['G3'].unique())

[1 2 5 3 4]
[ 6 10 15 11 19  9 12 14 16  5  8 17 18 13 20  7  0  4]


In [11]:
data['constant'] = 1

ave_grade_low_alc = data.loc[(data.Dalc == 1) | (data.Dalc == 2)].G3.mean()
ave_grade_high_alc = data.loc[(data.Dalc == 3) | (data.Dalc == 4) | (data.Dalc == 5)].G3.mean()

print("Average grade")
print("      drinkers: {:>0.1f}".format(ave_grade_high_alc))
print("  non-drinkers: {:>0.1f}".format(ave_grade_low_alc))

Average grade
      drinkers: 10.4
  non-drinkers: 10.8


In [12]:
ols = sm.OLS(data.G3, data[['constant', 'Dalc']])
res = ols.fit()
res.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
constant,11.2266,0.304,36.940,0.000,10.630,11.823
Dalc,-0.3334,0.172,-1.938,0.053,-0.671,0.004


In [13]:
est = res.params.Dalc
conf_ints = res.conf_int(alpha=0.05, cols=None)
lo, hi = conf_ints[0]['Dalc'], conf_ints[1]['Dalc']

print('            estimate   95% C.I.')
print('difference   {:>6.2f}   ({:>0.1f}, {:>0.1f})'.format(est, lo, hi))

            estimate   95% C.I.
difference    -0.33   (-0.7, 0.0)


In [14]:
""" 'sex', 'address', 'studytime', 'failures', 'schoolsup', 'famsup',
       'paid', 'activities', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3'
       """

summaries = OrderedDict((
    ('sex', lambda x: (100 * (x == 0)).mean()),
    ('address', lambda x: (100 * (x == 0)).mean()),
    ('studytime', 'mean'),
    ('failures', 'mean'),
    ('schoolsup', lambda x: (100 * (x == 0)).mean()),
    ('famsup', lambda x: (100 * (x == 0)).mean()),
    ('paid', lambda x: (100 * (x == 0)).mean()),
    ('activities', lambda x: (100 * (x == 0)).mean()),
    ('higher', lambda x: (100 * (x == 0)).mean()),
    ('internet', lambda x: (100 * (x == 0)).mean()),
    ('romantic', lambda x: 100 * x.mean()),
    ('famrel', 'mean'), 
    ('freetime', 'mean'), 
    ('goout', 'mean'), 
    ('health', 'mean'), 
    ('absences', 'mean')
))

table = data.groupby('Dalc').agg(summaries)
table.sort_index(ascending=False, inplace=True)
table = table.T

table.index = [
    'sex, % female',
    'address, % rural',
    'studytime, mean',
    'failures, mean',
    'schoolsup, % no schoolsup',
    'famsup, % no famsup',
    'paid, % no paid',
    'activities, % no activities',
    'higher, % no higher',
    'internet, % no internet',
    'romantic, % no romantic', 
    'famrel, mean', 
    'freetime, mean', 
    'goout, mean', 
    'health, mean', 
    'absences, mean'
]

table.style.format("{:>0.1f}")

Dalc,5,4,3,2,1
"sex, % female",11.8,38.9,24.4,47.5,67.8
"address, % rural",29.4,27.8,44.4,32.8,28.5
"studytime, mean",1.7,1.7,1.6,1.8,2.0
"failures, mean",0.4,0.4,0.5,0.4,0.3
"schoolsup, % no schoolsup",88.2,88.9,95.6,91.0,88.5
"famsup, % no famsup",17.6,50.0,57.8,38.5,38.5
"paid, % no paid",76.5,50.0,77.8,66.4,72.0
"activities, % no activities",29.4,55.6,57.8,54.1,51.3
"higher, % no higher",29.4,16.7,13.3,14.8,8.5
"internet, % no internet",11.8,22.2,26.7,23.0,24.3
